In [ ]:
from watea.watea_perfs import get_fleet_wise_energy_distribution
from core.plt_utils import plt_3d_df
from watea.watea_constants import *

In [ ]:
df = (
    get_fleet_wise_energy_distribution(False)
    .assign(soc=lambda df: df.index.get_level_values(2))
    .assign(temp=lambda df: df.index.get_level_values(1))
    .assign(odo=lambda df: df.index.get_level_values(0))
    .drop(columns=["duration"])
    .xs(15.0, level=1)
    .eval("power = energy_added / sec_duration")
    .query("energy_added < 500 & energy_added > 300 & sec_duration < 900 & power < 4 & power > 1.5 & temp < 35")
)
plt_3d_df(df, "soc", "odo", "energy_added", color="power", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)